In [3]:
from itertools import product
from tqdm import tqdm
ngrams = lambda a, n: list(zip(*[a[i:] for i in range(n)]))  # function that extract all the n grams in a given sequence


def Combinations(proteins, n):
    return list(product(proteins, repeat=n))


def get_spectrum_embeddings(Seq, combinations, n):
    kmers = ngrams(Seq, n)
    embedding = np.zeros(len(combinations))
    for ngram in kmers:
        index = combinations.index(ngram)
        embedding[index] += 1
    return embedding


def get_mismatch_embeddings(Seq, combinations, n):
    proteins = ['A', 'C', 'G', 'T']
    decompose_seq = ngrams(Seq, n)
    embedding = np.zeros(len(combinations))
    for kmer in decompose_seq:
        index = combinations.index(kmer)
        embedding[index] += 1
        kmer_seq = list(kmer)
        for ind, cur_protein in enumerate(kmer_seq):
            for protein in proteins:
                if protein != cur_protein:
                    mismatch_kmer = list(kmer_seq)
                    mismatch_kmer[ind] = protein
                    mismatch_kmer = tuple(mismatch_kmer)
                    index_ = combinations.index(mismatch_kmer)
                    embedding[index_] += 0.3
    return embedding


def get_gram_matrix(X1, X2=[]):

    n2 = len(X2)
    n1 = len(X1)
    if n2 == 0:
        gram_matrix = X1 @ X1.T
        gram_matrix_copy = X1 @ X1.T
        gram_matrix = gram_matrix.astype(np.float32)
        for i in range(n1):
            for j in range(n1):
                gram_matrix[i, j] /= (gram_matrix_copy[i, i] * gram_matrix_copy[j, j]) ** 0.5
        print('Gram Matrix Computed for X1')
        return gram_matrix
    else:
        gram_matrix = X1 @ X2.T
        gram_matrix = gram_matrix.astype(np.float32)
        gram_X1 = X1 @ X1.T
        gram_X2 = X2 @ X2.T

        for i in range(n1):
            for j in range(n2):
                gram_matrix[i, j] /= (gram_X2[j, j] * gram_X1[i, i]) ** 0.5
        print('Gram Matrix Computed for X2')
        return gram_matrix

In [9]:

def get_gram_matrix(X1, X2=[]):

    n2 = len(X2)
    n1 = len(X1)
    if n2 == 0:
        gram_matrix = X1 @ X1.T
        gram_matrix_copy = X1 @ X1.T
        gram_matrix = gram_matrix.astype(np.float32)
        for i in range(n1):
            for j in range(n1):
                gram_matrix[i, j] /= (gram_matrix_copy[i, i] * gram_matrix_copy[j, j]) ** 0.5
        print('Gram Matrix Computed for X1')
        return gram_matrix
    else:
        gram_matrix = X1 @ X2.T
        gram_matrix = gram_matrix.astype(np.float32)
        gram_X1 = X1 @ X1.T
        gram_X2 = X2 @ X2.T

        for i in range(n1):
            for j in range(n2):
                gram_matrix[i, j] /= (gram_X2[j, j] * gram_X1[i, i]) ** 0.5
        print('Gram Matrix Computed for X2')
        return gram_matrix

In [4]:
import pandas as pd
import numpy as np
import os

X_train_0 = (pd.read_csv('data/Xtr0.csv',header=None).values).tolist()
Y_train_0 = (pd.read_csv('data/Ytr0.csv',sep=',',index_col=0).values)
X_train_0 = (np.array(X_train_0)[1:,1]).tolist()
X_test_0 = (pd.read_csv('data/Xte0.csv',header=None).values).tolist()
X_test_0 = (np.array(X_test_0)[1:,1]).tolist()
Y_train_0[Y_train_0 == 0] = -1
from sklearn.model_selection import train_test_split



  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]


NameError: name 'j' is not defined

In [38]:
kernels_0 = {'mismatch_6': [6, get_mismatch_embeddings],
             'mismatch_7': [7, get_mismatch_embeddings],
            'spectrum_7': [7, get_spectrum_embeddings],
            'spectrum_6': [6, get_spectrum_embeddings]}

In [39]:
gram_matrices_0 = {}
for key in kernels_0.keys():
    train_filename = 'gram_matrices/train_' + key + '_dataset0.npy'
    test_filename = 'gram_matrices/test_' + key + '_dataset0.npy'
    length = kernels_0[key][0]
    embedding_func = kernels_0[key][1]
    DNA_combinations = Combinations(proteins=['A', 'C', 'G', 'T'], n=length)
    
    if path.exists(train_filename):
        print(train_filename, ' already exists !')
        gram_train_0 = np.load(train_filename)
    else:
        print('Creating ', train_filename)
        train_embeddings_0 = np.empty([len(X_train_0), len(DNA_combinations)])
        for i in tqdm(range(len(X_train_0))):
            train_embeddings_0[i, :] = embedding_func(Seq=X_train_0[i], combinations=DNA_combinations, n=length)
        gram_train_0 = get_gram_matrix(train_embeddings_0)
        np.save(train_filename, gram_train_0)
    if path.exists(test_filename):
        print(test_filename, ' already exists !')
        gram_test_0 = np.load(test_filename)
    else:
        print('Creating ', test_filename)
        test_embeddings_0 = np.empty([len(X_test_0), len(DNA_combinations)])
        for i in tqdm(range(len(X_test_0))):
            test_embeddings_0[i, :] = embedding_func(Seq=X_test_0[i], combinations=DNA_combinations, n=length)
        gram_test_0 = get_gram_matrix(train_embeddings_0, test_embeddings_0)
        np.save(test_filename, gram_test_0)
    gram_matrices_0[key] = {'train': gram_train_0,
                           'test': gram_test_0}

  0%|▎                                                                                | 7/2000 [00:00<00:28, 68.78it/s]

gram_matrices/train_mismatch_6_dataset0.npy  already exists !
gram_matrices/test_mismatch_6_dataset0.npy  already exists !
gram_matrices/train_mismatch_7_dataset0.npy  already exists !
gram_matrices/test_mismatch_7_dataset0.npy  already exists !
Creating  gram_matrices/train_spectrum_7_dataset0.npy


  1%|▌                                                                                | 7/1000 [00:00<00:15, 64.34it/s]

Gram Matrix Computed for X1
Creating  gram_matrices/test_spectrum_7_dataset0.npy


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:15<00:00, 66.23it/s]


Gram Matrix Computed for X2
gram_matrices/train_spectrum_6_dataset0.npy  already exists !
gram_matrices/test_spectrum_6_dataset0.npy  already exists !


In [14]:
import cvxopt
class SVMC:

    def __init__(self, c=1, min_sv=1e-4):
        self.alpha_ = None
        self.c = c  # corresponds to (1/2*lambda)
        # if y_train is not None: self.C = float(self.C)
        self.min_sv = min_sv

    def fit(self, kernel_train, label):
        n = label.shape[0]
        diag = np.zeros((n, n))
        np.fill_diagonal(diag, label)
        P = np.dot(diag, np.dot(kernel_train, diag))
        Pcvx = cvxopt.matrix(P)

        # Pcvx = cvxopt.matrix(np.outer(label,label) * kernel_train)
        qcvx = cvxopt.matrix(np.ones(n) * -1)

        if self.c is None:
            G = cvxopt.matrix(np.diag(np.ones(n) * -1))
            h = cvxopt.matrix(np.zeros(n))
        else:
            Ginf = np.diag(np.ones(n) * -1)
            Gsup = np.identity(n)
            G = cvxopt.matrix(np.vstack((Ginf, Gsup)))
            hinf = np.zeros(n)
            hsup = np.ones(n) * self.c
            h = cvxopt.matrix(np.hstack((hinf, hsup)))

        A = label.transpose()
        A = A.astype('double')
        Acvx = cvxopt.matrix(A)
        bcvx = cvxopt.matrix(0.0)

        # Solve QP problem using cvxopt solver for qp problems
        u = cvxopt.solvers.qp(Pcvx, qcvx, G, h, Acvx, bcvx)

        # take Lagrange multipliers, and the solution of the dual problem
        alpha = np.ravel(u['x'])

        sv = alpha > self.min_sv
        ind = np.arange(len(alpha))[sv]

        self.alpha_ = alpha[sv]
        self.sv = np.argwhere(sv == True)
        self.sv_label = label[sv]
        print("%d support vectors out of %d points" % (len(self.alpha_), n))

        # Bias value/intercept
        self.b = 0 * 1.0;
        # self.b = self.b.astype(np.float64)
        for i in range(len(self.alpha_)):
            self.b += self.sv_label[i]
            self.b -= np.sum(self.alpha_ * self.sv_label[:, 0] * kernel_train[sv, ind[i]])
        self.b /= len(self.alpha_)

    def get_coef(self):
        return list(self.alpha_)

    def predict(self, kernel_test):
        
        y_predict = np.zeros(kernel_test.shape[1])

        for i in range(kernel_test.shape[1]):
            y_predict[i] = sum(alpha * sv_label * kernel_test[sv, i] for alpha, sv, sv_label in
                               zip(self.alpha_, self.sv, self.sv_label[:, 0]))
        return y_predict + self.b

        prediction = np.sign(y_predict + self.b)

        return prediction

    def predict_class(self, kernel_test):
        
        prediction = np.array(self.predict(kernel_test) >= 0, dtype=int)
        prediction[prediction == 0] = -1
        return prediction

In [49]:
c = 0.5
sv = 1e-4
lambda_log_reg = 1
tolerance = 0.001
list_kernels = list(gram_matrices_0.keys()) * 4
list_of_prediction_test_0 = []
from sklearn.metrics import accuracy_score
for kernel in list_kernels:
    print(f'Predicting with {kernel}')
    gram_train_0 = gram_matrices_0[kernel]['train']
    gram_test_0 = gram_matrices_0[kernel]['test']
    test_size = 0.2
    list_train, list_val = train_test_split(list(range(2000)), test_size=test_size)
    gram_train = gram_train_0[list_train, :][:, list_train]
    gram_val = gram_train_0[list_train, :][:, list_val]
    y_train_split_0 = Y_train_0[list_train]
    y_val_split_0 = Y_train_0[list_val]

    svm_test = SVMC(c=c, min_sv=sv)
    svm_test.fit(gram_train, y_train_split_0)
    y_val_pred = svm_test.predict_class(gram_val).reshape(-1)
    print('Val Accuracy =', accuracy_score(y_val_split_0.reshape(-1), y_val_pred))
    y_test_pred_0 = svm_test.predict_class(gram_test_0[list_train, :])
    y_test_pred_0[y_test_pred_0 == -1] = 0
    list_of_prediction_test_0.append(y_test_pred_0)

y_pred_0 = np.array(np.array(list_of_prediction_test_0).mean(axis=0).reshape((-1,))>0.5,dtype=int)


Predicting with mismatch_6
     pcost       dcost       gap    pres   dres
 0: -7.8991e+02 -1.9461e+03  1e+04  5e+00  5e-15
 1: -4.5130e+02 -1.5863e+03  1e+03  8e-02  4e-15
 2: -5.0523e+02 -6.5212e+02  2e+02  1e-02  3e-15
 3: -5.5555e+02 -5.9959e+02  4e+01  2e-03  3e-15
 4: -5.7070e+02 -5.8451e+02  1e+01  4e-04  3e-15
 5: -5.7555e+02 -5.7857e+02  3e+00  6e-05  3e-15
 6: -5.7680e+02 -5.7707e+02  3e-01  4e-15  4e-15
 7: -5.7692e+02 -5.7693e+02  1e-02  1e-14  4e-15
 8: -5.7693e+02 -5.7693e+02  2e-04  3e-14  4e-15
Optimal solution found.
1393 support vectors out of 1600 points
Val Accuracy = 0.625
Predicting with mismatch_7
     pcost       dcost       gap    pres   dres
 0: -6.5706e+02 -1.9312e+03  1e+04  5e+00  2e-15
 1: -4.2561e+02 -1.5679e+03  1e+03  2e-01  2e-15
 2: -4.6688e+02 -6.0891e+02  1e+02  1e-02  1e-15
 3: -5.1381e+02 -5.6384e+02  5e+01  4e-03  1e-15
 4: -5.3210e+02 -5.4640e+02  1e+01  7e-04  1e-15
 5: -5.3736e+02 -5.4021e+02  3e+00  1e-04  2e-15
 6: -5.3862e+02 -5.3875e+02  1

 8: -5.3811e+02 -5.3811e+02  8e-05  4e-11  2e-15
Optimal solution found.
1437 support vectors out of 1600 points
Val Accuracy = 0.6425
Predicting with spectrum_7
     pcost       dcost       gap    pres   dres
 0: -5.6145e+02 -1.9614e+03  1e+04  5e+00  2e-15
 1: -4.0497e+02 -1.5934e+03  1e+03  2e-01  1e-15
 2: -4.4093e+02 -5.9117e+02  2e+02  2e-02  1e-15
 3: -4.9405e+02 -5.3116e+02  4e+01  3e-03  9e-16
 4: -5.0839e+02 -5.1563e+02  7e+00  7e-05  9e-16
 5: -5.1085e+02 -5.1169e+02  8e-01  4e-06  9e-16
 6: -5.1118e+02 -5.1122e+02  4e-02  1e-07  1e-15
 7: -5.1120e+02 -5.1120e+02  1e-03  3e-09  1e-15
 8: -5.1120e+02 -5.1120e+02  6e-05  6e-11  1e-15
Optimal solution found.
1490 support vectors out of 1600 points
Val Accuracy = 0.6625
Predicting with spectrum_6
     pcost       dcost       gap    pres   dres
 0: -6.4183e+02 -1.9371e+03  1e+04  5e+00  2e-15
 1: -4.2126e+02 -1.5785e+03  2e+03  2e-01  1e-15
 2: -4.5792e+02 -6.0583e+02  2e+02  2e-02  1e-15
 3: -5.1304e+02 -5.5262e+02  4e+01  3e-03

In [42]:

X_train_1 = (pd.read_csv('data/Xtr1.csv',header=None).values).tolist()
Y_train_1 = (pd.read_csv('data/Ytr1.csv',sep=',',index_col=0).values)
X_train_1 = (np.array(X_train_1)[1:,1]).tolist()
X_test_1 = (pd.read_csv('data/Xte1.csv',header=None).values).tolist()
X_test_1 = (np.array(X_test_1)[1:,1]).tolist()
Y_train_1[Y_train_1 == 0] = -1
from sklearn.model_selection import train_test_split

In [43]:
kernels_1 = {'mismatch_6': [6, get_mismatch_embeddings],
             'mismatch_7': [7, get_mismatch_embeddings],
            'spectrum_7': [7, get_spectrum_embeddings],
            'spectrum_6': [6, get_spectrum_embeddings]}

gram_matrices_1 = {}
for key in kernels_1.keys():
    train_filename = 'gram_matrices/train_' + key + '_dataset1.npy'
    test_filename = 'gram_matrices/test_' + key + '_dataset1.npy'
    length = kernels_1[key][0]
    embedding_func = kernels_1[key][1]
    DNA_combinations = Combinations(proteins=['A', 'C', 'G', 'T'], n=length)
    
    if path.exists(train_filename):
        print(train_filename, ' already exists !')
        gram_train_1 = np.load(train_filename)
    else:
        print('Creating ', train_filename)
        train_embeddings_1 = np.empty([len(X_train_1), len(DNA_combinations)])
        for i in tqdm(range(len(X_train_1))):
            train_embeddings_1[i, :] = embedding_func(Seq=X_train_1[i], combinations=DNA_combinations, n=length)
        gram_train_1 = get_gram_matrix(train_embeddings_1)
        np.save(train_filename, gram_train_1)
    if path.exists(test_filename):
        print(test_filename, ' already exists !')
        gram_test_1 = np.load(test_filename)
    else:
        print('Creating ', test_filename)
        test_embeddings_1 = np.empty([len(X_test_1), len(DNA_combinations)])
        for i in tqdm(range(len(X_test_1))):
            test_embeddings_1[i, :] = embedding_func(Seq=X_test_1[i], combinations=DNA_combinations, n=length)
        gram_test_1 = get_gram_matrix(train_embeddings_1, test_embeddings_1)
        np.save(test_filename, gram_test_1)
    gram_matrices_1[key] = {'train': gram_train_1,
                           'test': gram_test_1}

  0%|                                                                                 | 1/2000 [00:00<03:46,  8.83it/s]

Creating  gram_matrices/train_mismatch_6_dataset1.npy


  0%|▏                                                                                | 2/1000 [00:00<01:18, 12.77it/s]

Gram Matrix Computed for X1
Creating  gram_matrices/test_mismatch_6_dataset1.npy


  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

...

Gram Matrix Computed for X2
Creating  gram_matrices/train_spectrum_6_dataset1.npy


  2%|█▉                                                                             | 24/1000 [00:00<00:04, 238.26it/s]

Gram Matrix Computed for X1
Creating  gram_matrices/test_spectrum_6_dataset1.npy


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 258.68it/s]


Gram Matrix Computed for X2


In [50]:
c = 0.5
sv = 1e-4
tolerance = 0.001
list_kernels = list(gram_matrices_1.keys()) * 4
list_of_prediction_test_1 = []
from sklearn.metrics import accuracy_score
for kernel in list_kernels:
    print(f'Predicting with {kernel}')
    gram_train_1 = gram_matrices_1[kernel]['train']
    gram_test_1 = gram_matrices_1[kernel]['test']
    test_size = 0.2
    list_train, list_val = train_test_split(list(range(2000)), test_size=test_size)
    gram_train = gram_train_1[list_train, :][:, list_train]
    gram_val = gram_train_1[list_train, :][:, list_val]
    y_train_split_1 = Y_train_1[list_train]
    y_val_split_1 = Y_train_1[list_val]

    svm_test = SVMC(c=c, min_sv=sv)
    svm_test.fit(gram_train, y_train_split_1)
    y_val_pred = svm_test.predict_class(gram_val).reshape(-1)
    print('Val Accuracy =', accuracy_score(y_val_split_1.reshape(-1), y_val_pred))
    y_test_pred_1 = svm_test.predict_class(gram_test_1[list_train, :])
    y_test_pred_1[y_test_pred_1 == -1] = 0
    list_of_prediction_test_1.append(y_test_pred_1)

y_pred_1 = np.array(np.array(list_of_prediction_test_1).mean(axis=0).reshape((-1,))>0.5,dtype=int)


Predicting with mismatch_6
     pcost       dcost       gap    pres   dres
 0: -7.7837e+02 -1.9857e+03  1e+04  5e+00  5e-15
 1: -4.5182e+02 -1.6217e+03  1e+03  4e-03  4e-15
 2: -5.1719e+02 -6.5355e+02  1e+02  5e-04  3e-15
 3: -5.7019e+02 -6.0165e+02  3e+01  9e-05  3e-15
 4: -5.8335e+02 -5.8957e+02  6e+00  1e-05  3e-15
 5: -5.8617e+02 -5.8689e+02  7e-01  1e-06  3e-15
 6: -5.8651e+02 -5.8657e+02  6e-02  1e-07  3e-15
 7: -5.8654e+02 -5.8654e+02  2e-03  2e-09  3e-15
 8: -5.8654e+02 -5.8654e+02  5e-05  3e-11  4e-15
Optimal solution found.
1454 support vectors out of 1600 points
Val Accuracy = 0.625
Predicting with mismatch_7
     pcost       dcost       gap    pres   dres
 0: -6.4784e+02 -1.8413e+03  1e+04  5e+00  2e-15
 1: -4.3341e+02 -1.4746e+03  1e+03  3e-15  2e-15
 2: -4.9271e+02 -6.0813e+02  1e+02  2e-16  2e-15
 3: -5.3930e+02 -5.6261e+02  2e+01  2e-15  2e-15
 4: -5.4939e+02 -5.5318e+02  4e+00  2e-16  2e-15
 5: -5.5123e+02 -5.5151e+02  3e-01  6e-15  2e-15
 6: -5.5137e+02 -5.5138e+02  1

 2: -4.9551e+02 -5.9767e+02  1e+02  6e-15  1e-15
 3: -5.4000e+02 -5.6029e+02  2e+01  2e-14  9e-16
 4: -5.4961e+02 -5.5210e+02  2e+00  1e-14  9e-16
 5: -5.5089e+02 -5.5106e+02  2e-01  4e-15  9e-16
 6: -5.5098e+02 -5.5099e+02  6e-03  4e-15  1e-15
 7: -5.5098e+02 -5.5098e+02  3e-04  6e-15  1e-15
Optimal solution found.
1578 support vectors out of 1600 points
Val Accuracy = 0.6325
Predicting with spectrum_6
     pcost       dcost       gap    pres   dres
 0: -6.3395e+02 -1.8424e+03  1e+04  5e+00  2e-15
 1: -4.2840e+02 -1.4757e+03  1e+03  7e-15  1e-15
 2: -4.8664e+02 -6.0293e+02  1e+02  1e-14  2e-15
 3: -5.3225e+02 -5.5894e+02  3e+01  2e-14  1e-15
 4: -5.4425e+02 -5.4748e+02  3e+00  7e-15  1e-15
 5: -5.4571e+02 -5.4602e+02  3e-01  2e-16  1e-15
 6: -5.4586e+02 -5.4587e+02  1e-02  4e-15  1e-15
 7: -5.4587e+02 -5.4587e+02  4e-04  1e-14  1e-15
Optimal solution found.
1522 support vectors out of 1600 points
Val Accuracy = 0.67


In [45]:

X_train_2 = (pd.read_csv('data/Xtr2.csv',header=None).values).tolist()
Y_train_2 = (pd.read_csv('data/Ytr2.csv',sep=',',index_col=0).values)
X_train_2 = (np.array(X_train_2)[1:,1]).tolist()
X_test_2 = (pd.read_csv('data/Xte2.csv',header=None).values).tolist()
X_test_2 = (np.array(X_test_2)[1:,1]).tolist()
Y_train_2[Y_train_2 == 0] = -1
from sklearn.model_selection import train_test_split

In [46]:
kernels_2 = {'mismatch_6': [6, get_mismatch_embeddings],
             'mismatch_7': [7, get_mismatch_embeddings],
            'spectrum_7': [7, get_spectrum_embeddings],
            'spectrum_6': [6, get_spectrum_embeddings]}

gram_matrices_2 = {}
for key in kernels_2.keys():
    train_filename = 'gram_matrices/train_' + key + '_dataset2.npy'
    test_filename = 'gram_matrices/test_' + key + '_dataset2.npy'
    length = kernels_2[key][0]
    embedding_func = kernels_2[key][1]
    DNA_combinations = Combinations(proteins=['A', 'C', 'G', 'T'], n=length)
    
    if path.exists(train_filename):
        print(train_filename, ' already exists !')
        gram_train_2 = np.load(train_filename)
    else:
        print('Creating ', train_filename)
        train_embeddings_2 = np.empty([len(X_train_2), len(DNA_combinations)])
        for i in tqdm(range(len(X_train_2))):
            train_embeddings_2[i, :] = embedding_func(Seq=X_train_2[i], combinations=DNA_combinations, n=length)
        gram_train_2 = get_gram_matrix(train_embeddings_2)
        np.save(train_filename, gram_train_2)
    if path.exists(test_filename):
        print(test_filename, ' already exists !')
        gram_test_2 = np.load(test_filename)
    else:
        print('Creating ', test_filename)
        test_embeddings_2 = np.empty([len(X_test_2), len(DNA_combinations)])
        for i in tqdm(range(len(X_test_2))):
            test_embeddings_2[i, :] = embedding_func(Seq=X_test_2[i], combinations=DNA_combinations, n=length)
        gram_test_2 = get_gram_matrix(train_embeddings_2, test_embeddings_2)
        np.save(test_filename, gram_test_2)
    gram_matrices_2[key] = {'train': gram_train_2,
                           'test': gram_test_2}

  0%|                                                                                 | 2/2000 [00:00<02:37, 12.67it/s]

Creating  gram_matrices/train_mismatch_6_dataset2.npy


  0%|▏                                                                                | 2/1000 [00:00<01:13, 13.64it/s]

Gram Matrix Computed for X1
Creating  gram_matrices/test_mismatch_6_dataset2.npy


  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

...

Gram Matrix Computed for X2
Creating  gram_matrices/train_spectrum_6_dataset2.npy


  3%|██                                                                             | 26/1000 [00:00<00:03, 255.58it/s]

Gram Matrix Computed for X1
Creating  gram_matrices/test_spectrum_6_dataset2.npy


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 259.33it/s]


Gram Matrix Computed for X2


In [52]:
c = 0.5
sv = 1e-4
tolerance = 0.001
list_kernels = list(gram_matrices_2.keys()) * 4
list_of_prediction_test_2 = []
from sklearn.metrics import accuracy_score
for kernel in list_kernels:
    print(f'Predicting with {kernel}')
    gram_train_2 = gram_matrices_2[kernel]['train']
    gram_test_2 = gram_matrices_2[kernel]['test']
    test_size = 0.2
    list_train, list_val = train_test_split(list(range(2000)), test_size=test_size)
    gram_train = gram_train_2[list_train, :][:, list_train]
    gram_val = gram_train_2[list_train, :][:, list_val]
    y_train_split_2 = Y_train_2[list_train]
    y_val_split_2 = Y_train_2[list_val]

    svm_test = SVMC(c=c, min_sv=sv)
    svm_test.fit(gram_train, y_train_split_2)
    y_val_pred = svm_test.predict_class(gram_val).reshape(-1)
    print('Val Accuracy =', accuracy_score(y_val_split_2.reshape(-1), y_val_pred))
    y_test_pred_2 = svm_test.predict_class(gram_test_2[list_train, :])
    y_test_pred_2[y_test_pred_2 == -1] = 0
    list_of_prediction_test_2.append(y_test_pred_2)

y_pred_2 = np.array(np.array(list_of_prediction_test_2).mean(axis=0).reshape((-1,))>0.5,dtype=int)


Predicting with mismatch_6
     pcost       dcost       gap    pres   dres
 0: -6.4781e+02 -1.9684e+03  1e+04  5e+00  4e-15
 1: -3.7736e+02 -1.6005e+03  2e+03  2e-01  3e-15
 2: -4.0539e+02 -5.6745e+02  2e+02  2e-02  2e-15
 3: -4.5789e+02 -5.0382e+02  5e+01  3e-03  2e-15
 4: -4.7296e+02 -4.8600e+02  1e+01  7e-04  2e-15
 5: -4.7786e+02 -4.8015e+02  2e+00  7e-05  2e-15
 6: -4.7881e+02 -4.7899e+02  2e-01  5e-06  2e-15
 7: -4.7889e+02 -4.7890e+02  8e-03  2e-07  2e-15
 8: -4.7889e+02 -4.7889e+02  3e-04  5e-09  2e-15
Optimal solution found.
1234 support vectors out of 1600 points
Val Accuracy = 0.7225
Predicting with mismatch_7
     pcost       dcost       gap    pres   dres
 0: -5.2971e+02 -1.8561e+03  1e+04  5e+00  2e-15
 1: -3.5248e+02 -1.4881e+03  1e+03  2e-01  2e-15
 2: -3.7969e+02 -5.2006e+02  1e+02  2e-02  1e-15
 3: -4.2523e+02 -4.6727e+02  4e+01  3e-03  1e-15
 4: -4.4002e+02 -4.4978e+02  1e+01  3e-04  1e-15
 5: -4.4339e+02 -4.4471e+02  1e+00  2e-05  1e-15
 6: -4.4389e+02 -4.4399e+02  

 6: -4.3138e+02 -4.3163e+02  2e-01  2e-14  2e-15
 7: -4.3148e+02 -4.3150e+02  2e-02  7e-15  2e-15
 8: -4.3149e+02 -4.3149e+02  7e-04  2e-14  2e-15
 9: -4.3149e+02 -4.3149e+02  2e-05  3e-14  2e-15
Optimal solution found.
1254 support vectors out of 1600 points
Val Accuracy = 0.71
Predicting with spectrum_7
     pcost       dcost       gap    pres   dres
 0: -4.8383e+02 -1.8505e+03  1e+04  5e+00  2e-15
 1: -3.6240e+02 -1.4844e+03  1e+03  2e-01  9e-16
 2: -3.9232e+02 -5.3143e+02  1e+02  1e-02  8e-16
 3: -4.3772e+02 -4.7269e+02  4e+01  3e-03  8e-16
 4: -4.4999e+02 -4.5693e+02  7e+00  2e-04  8e-16
 5: -4.5239e+02 -4.5339e+02  1e+00  3e-05  8e-16
 6: -4.5275e+02 -4.5289e+02  1e-01  3e-06  8e-16
 7: -4.5280e+02 -4.5281e+02  7e-03  1e-07  9e-16
 8: -4.5281e+02 -4.5281e+02  2e-04  2e-09  9e-16
Optimal solution found.
1407 support vectors out of 1600 points
Val Accuracy = 0.76
Predicting with spectrum_6
     pcost       dcost       gap    pres   dres
 0: -5.1918e+02 -1.8687e+03  1e+04  5e+00  2e

In [53]:
y_pred = list(y_pred_0) + list(y_pred_1) + list(y_pred_2)
with open("outputs/ensembling_kernels_16.csv", 'w') as f:
    f.write('Id,Bound\n')
    for i in range(len(y_pred)):
        f.write(str(i)+','+str(y_pred[i])+'\n')